# Гиперпицца

In [1]:
from html.parser import HTMLParser
from numpy import array as Arr
import pandas as pd
import os, io, re
import PySimpleGUI as sg

take_money_list, comp_cr2_dict = [], {}
emp_hr_count2_dict, make_deal_dict = {}, {}
emp_hr_total_dict, emp_cost_max = {}, {}
emp_hr_count3_dict, emp_hr_dict = {}, {}
comp_cr4_dict, emp_hr_count4_dict = {}, {}

## Preprocess text
class MyHTMLParserHP(HTMLParser):
    def handle_data(self, data):
        if not re.match(r'^\s*$', data):
            result.append(data)

## Get table and result list
def table_result_score(filepath):
    global result
    table_new, result = [], []

    parser = MyHTMLParserHP()
    with io.open(filepath, encoding = 'utf-8') as file:
        for line in file:
            parser.feed(line)
    
    global player_name
    #player_name = [line for line in result if line.startswith('Игрок')][0][7:]
    score = float([line for line in result if line.startswith('Баланс')][0][8:])
    table = result[result.index('Таблица транзакций') + 3 : result.index('Описание транзакций')]
    for i in range(0, len(table), 2):
        table_new.append((table[i], table[i + 1]))

    del result[:result.index('Описание транзакций') + 1]
    levels = [i for i, x in enumerate(result) if x[-2:] == ': ']
    #result = [result[i].lower() for i in range(len(result))]
    result = [result[i].lower() + result[i + 1].lower() if i in levels else result[i] for i in range(len(result))]
    result = [result[i] for i in range(len(result)) if i - 1 not in levels]
    indexs = [i for i, x in enumerate(result) if x == 'выполнена']
    indexs.append(len(result))
    result = [result[indexs[n]:indexs[n + 1]] for n in range(len(indexs)-1)]
    result = [x if 'уровень игры: ' in ''.join(x) else \
            x.append('уровень игры: ' + table[table.index(x[1])+1]) or x for x in result]
    result = [x[:x.index('отклонена')] if 'отклонена' in x else x for x in result]
    result = [x[:x.index('ожидает одобрения участников')] if 'ожидает одобрения участников' in x else x for x in result]
    return(table_new, result, score)

## Rules Function
def rules(code, filename, marks):
    numbers = Arr([0]*8)
    if code == "1.1":
        if ('зафиксировать инженерам распределение приза от корпорации X', '1') in tables[filename]:
            take_money_list.append(filename)
            numbers += Arr([0,1,1,0,0,0,0,0])
    elif code == "1.2":
        if ('подарить талеры', '1') in tables[filename] \
                and any([y[y.find('кто')+5:] in ' '.join(take_money_list).lower() \
                for y in sum([x for x in results[filename] if 'подарить талеры' in x \
                and 'уровень игры: 1' in x], []) if y.startswith('кто')]):
            numbers += Arr([0,1,0,0,0,0,0,0])
    elif code == "1.3":
        if ('взять у скаута инструмент в аренду', '1') in tables[filename] \
                and any([y for y in sum([x for x in results[filename] if 'взять у скаута инструмент в аренду' in x \
                and 'уровень игры: 1' in x], []) if y.startswith('cкаут')]): # english c
            numbers += Arr([0,1,1,0,0,0,0,0])
    elif code == "1.4":
        if ('покупка готовой продукции', '2') in tables[filename] or \
                any([y for y in sum([x for x in results[filename] if 'взять у скаута инструмент в аренду' in x \
                and 'уровень игры: 1' in x], []) if y.startswith('инженер')]):
            numbers += Arr([2,0,0,0,0,0,0,0])
    elif code == "1.5":
        if filename in ' '.join(take_money_list) \
                and all([('подарить талеры', k) not in tables[filename] for k in ['1', '2']]):
            numbers += Arr([0,0,1,1,0,0,0,0])
    elif code == "1.6":
        if filename in ' '.join(take_money_list) \
                and any([('подарить талеры', k) in tables[filename] for k in ['1', '2']]):
            numbers += Arr([0,0,0,0,0,1,0,0])
    elif code == "2.0":
        for comp_name in [y[y.find('название')+10:] \
                for y in sum([x for x in results[filename] if 'создать компанию' in x \
                and 'уровень игры: 2' in x], []) if y.startswith('название')]:
            comp_cr2_dict[comp_name].append(filename) if comp_name in comp_cr2_dict.keys() \
                    else comp_cr2_dict.update({comp_name: [filename]})
    elif code == "2.1":
        if all([(action, '2') in tables[filename] for action in \
                ['создать компанию', 'перевести талеры на счет компании', 'нанять сотрудника']]):
            numbers += Arr([0,0,1,0,0,0,0,0])
    elif code == "2.2":
        if ('перевести талеры на счет компании', '2') in tables[filename] \
                and max([len(x) for x in comp_cr2_dict.values() if filename in x]) > 1:
            numbers += Arr([0,0,1,0,0,1,0,0])
    elif code == "2.3":
        if any([y for y in sum([x for x in results[filename] if 'тип договора: продажа инструмента' in x \
                and 'уровень игры: 2' in x], [])]): # english c
            numbers += Arr([0,1,1,0,0,0,0,0])
    elif code == "2.4":
        if scores[filename] <= -400 and tables[filename].count(('нанять сотрудника', '2')) < 5:
            numbers += Arr([-1,0,-1,0,0,0,0,0])
    elif code == "2.5":
        if scores[filename] <= -400 and tables[filename].count(('нанять сотрудника', '2')) >= 5:
            numbers += Arr([-1,0,-1,0,0,1,0,0])
    elif code == "2.6":
        emp_hr_count2_dict[filename] = len([y[y.find('ceo')+5:] in filename.lower() \
                for y in sum([x for x in results[filename] if 'нанять сотрудника' in x \
                and 'уровень игры: 2' in x], []) if y.startswith('ceo')]) - \
                len([y for y in sum([x for x in results[filename] if 'уволить сотрудника' in x \
                and 'уровень игры: 2' in x], [])])
        emp_cost_max[filename[7:-5].lower()] = max([int(y[y.find('зарплата')+10:]) \
                for y in sum([x for x in results[filename] if 'нанять сотрудника' in x and 'уровень игры: 2' in x \
                and any(['инженер: '+filename[7:-5].lower() in z for z in x])], []) if y.startswith('зарплата')], default=0)
        if emp_hr_count2_dict[filename] > 3:
            numbers += Arr([0,1,1,0,0,0,0,0])
    elif code == "2.7":
        if ('покупка готовой продукции', '2') in tables[filename] \
                and len([x for x in results[filename] if 'от компании получен  продукт(услуга): отдел закупок корпорации 1'
                in x or 'от компании получен  продукт(услуга): отдел закупок корпорации 2' in x]):
            numbers += Arr([0,1,1,0,0,0,0,0])
    elif code == "2.8":
        if any([filename in x for x in comp_cr2_dict.values()]) \
                and max([len(x) for x in comp_cr2_dict.values() if filename in x]) > 1 \
                and ('перевести талеры на счет компании', '2') not in tables[filename]:
            numbers += Arr([0,0,-2,0,0,0,1,0])
    elif code == "2.9":
        if any([y[y.find('ceo')+5:] not in filename.lower() \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x \
                and 'уровень игры: 2' in x], []) if y.startswith('ceo')]):
            numbers += Arr([1,0,0,1,0,0,0,0])
        make_deal_dict[filename] = list(set([y[y.find('тип договора')+14:] \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x and 'уровень игры: 2' in x \
                and all([filename[7:-5].lower() not in z for z in x])], []) if y.startswith('тип договора')]))
    elif code == "3.1":
        if any([y[y.find('инвестор')+10:] not in filename.lower() \
                for y in sum([x for x in results[filename] if 'инвестировать' in x \
                and 'уровень игры: 3' in x], []) if y.startswith('инвестор')]):
            numbers += Arr([0,0,0,2,0,0,0,0])
    elif code == "3.2":
        make_deal_x = ([y[y.find('тип договора')+14:] \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x and 'уровень игры: 3' in x \
                and all([filename[7:-5].lower() not in z for z in x])], []) if y.startswith('тип договора')])
        if any([y not in make_deal_dict[filename]+['материалы'] for y in make_deal_x]) or make_deal_x.count('материалы') > 2:
            numbers += Arr([1,0,0,2,3,0,0,0])
    elif code == "3.3":
        if ('нанять сотрудника', '3') in tables[filename]:
            numbers += Arr([0,0,0,0,0,0,0,0])
    elif code == "3.4":
        emp_hr_count3_dict[filename] = len([y[y.find('ceo')+5:] in filename.lower() \
                for y in sum([x for x in results[filename] if 'нанять сотрудника' in x \
                and 'уровень игры: 3' in x], []) if y.startswith('ceo')]) - \
                len([y for y in sum([x for x in results[filename] if 'уволить сотрудника' in x \
                and 'уровень игры: 3' in x], [])])
        if 0 < emp_hr_count3_dict[filename] < min(3, emp_hr_count2_dict[filename]):
            numbers += Arr([0,1,1,0,0,0,0,0])
    # 3.5! 3.6!
    elif code == "3.7":
        if any([y[y.find('ceo')+5:] in filename.lower() \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x \
                and all(['от компании получен  продукт(услуга): компания х1' not in x, \
                         'от компании получен  продукт(услуга): компания х2' not in x]) \
                and 'уровень игры: 3' in x], []) if y.startswith('ceo')]):
            numbers += Arr([1,1,1,0,0,0,0,0])
    elif code == "3.8":
        make_deal_dict[filename] += ([y[y.find('тип договора')+14:] \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x and 'уровень игры: 3' in x \
                and all([filename[7:-5].lower() not in z for z in x])], []) if y.startswith('тип договора')])
        if len(make_deal_dict[filename]) != len(set(make_deal_dict[filename])):
            numbers += Arr([0,0,1,1,0,0,0,0])
        make_deal_dict[filename] = list(set(make_deal_dict[filename]))
    elif code == "3.9":
        emp_hr_total_dict[filename] = [y[y.find('инженер')+9:] \
                for y in sum([x for x in results[filename] if 'нанять сотрудника' in x \
                and any(['ceo: '+filename[7:-5].lower() in z for z in x])], []) if y.startswith('инженер')]
        if sum([y[y.find('покупатель')+12:] not in [filename[7:-5].lower()]+emp_hr_total_dict[filename] \
                for y in sum([x for x in results[filename] if 'купить продукт B2C' in x \
                and 'уровень игры: 3' in x], []) if y.startswith('покупатель')]) >= 2:
            numbers += Arr([3,0,0,0,3,0,0,0])
    elif code == "3.10":
        if len([y[y.find('инвестор')+10:] not in filename.lower() \
                for y in sum([x for x in results[filename] if 'инвестировать' in x \
                and 'уровень игры: 3' in x], []) if y.startswith('инвестор')]) > 1:
            numbers += Arr([0,0,1,0,1,0,0,0])
    elif code == "3.11":
        if any([y[y.find('ceo')+5:] not in filename.lower() \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x \
                and any(['от компании получен  продукт(услуга): компания х1' in x, \
                         'от компании получен  продукт(услуга): компания х2' in x]) \
                and 'уровень игры: 3' in x], []) if y.startswith('ceo')]):
            numbers += Arr([0,0,1,0,0,0,0,0])
    elif code == "3.12":
        emp_hr_dict[filename] = [y[y.find('инженер')+9:] \
                for y in sum([x for x in results[filename] if 'нанять сотрудника' in x and 'уровень игры: 2' in x \
                and any(['ceo: '+filename[7:-5].lower() in z for z in x])], []) if y.startswith('инженер')]
        emp_hr_x = [dict([xx.split(': ') for xx in x if ':' in xx]) for x in results[filename] if 'нанять сотрудника' in x \
                and 'уровень игры: 3' in x and any(['ceo: '+filename[7:-5].lower() in z for z in x]) \
                and all(['инженер: '+emp not in x for emp in emp_hr_dict[filename]])]
        if any([int(_dict['зарплата']) > emp_cost_max[_dict['инженер']] for _dict in emp_hr_x]):
            numbers += Arr([0,0,1,0,0,0,0,0])
    elif code == "3.13":
        emp_hr_x = [dict([xx.split(': ') for xx in x if ':' in xx]) for x in results[filename] if 'нанять сотрудника' in x \
                and 'уровень игры: 3' in x and any(['ceo: '+filename[7:-5].lower() in z for z in x]) \
                and any(['инженер: '+emp in x for emp in emp_hr_dict[filename]])]
        if any([int(_dict['зарплата']) > emp_cost_max[_dict['инженер']] for _dict in emp_hr_x]):
            numbers += Arr([0,0,1,0,0,0,0,0])
    elif code == "3.14":
        revenue_x = [dict([xx.split(': ') for xx in x if ':' in xx]) for x in results[filename] if 'заключить сделки' in x \
            and 'уровень игры: 3' in x and any(['ceo: '+filename[7:-5].lower() in z for z in x])]
        if 0 < emp_hr_count3_dict[filename] < 3 and all([(float(x['цена'])*int(x['кол-во'])) > 10 for x in revenue_x]):
            numbers += Arr([0,0,0,0,0,3,0,0])
    elif code == "4.0":
        if any([y[y.find('инвестор')+10:] not in filename.lower() \
                for y in sum([x for x in results[filename] if 'инвестировать' in x \
                and 'уровень игры: 4' in x], []) if y.startswith('инвестор')]): #3.1 copy
            numbers += Arr([0,0,2,2,0,0,0,0])
        if len([y[y.find('инвестор')+10:] not in filename.lower() \
                for y in sum([x for x in results[filename] if 'инвестировать' in x \
                and 'уровень игры: 4' in x], []) if y.startswith('инвестор')]) > 1: #3.10 copy
            numbers += Arr([0,0,1,0,1,0,0,0])
        for comp_name in [y[y.find('название')+10:] \
                for y in sum([x for x in results[filename] if 'создать компанию' in x \
                and 'уровень игры: 4' in x], []) if y.startswith('название')]:
            comp_cr4_dict[comp_name].append(filename) if comp_name in comp_cr2_dict.keys() \
                    else comp_cr2_dict.update({comp_name: [filename]})
    elif code == "4.1":
        if ('закрыть компанию', '4') in tables[filename] and \
                max([len(x) for x in comp_cr4_dict.values() if filename in x], default=0) > 1:
            numbers += Arr([0,0,1,0,0,1,0,0])
    elif code == "4.2":
        make_deal_dict[filename] += ([y[y.find('тип договора')+14:] \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x and 'уровень игры: 4' in x \
                and all([filename[7:-5].lower() not in z for z in x])], []) if y.startswith('тип договора')])
        if len(make_deal_dict[filename]) != len(set(make_deal_dict[filename])):
            numbers += Arr([0,0,1,1,0,0,0,0])
    elif code == "4.3":
        emp_hr_count4_dict[filename] = len([y[y.find('ceo')+5:] in filename.lower() \
                for y in sum([x for x in results[filename] if 'нанять сотрудника' in x \
                and 'уровень игры: 4' in x], []) if y.startswith('ceo')]) - \
                len([y for y in sum([x for x in results[filename] if 'уволить сотрудника' in x \
                and 'уровень игры: 4' in x], [])])
        if 0 < emp_hr_count4_dict[filename] < min(3, emp_hr_count3_dict[filename]):
            numbers += Arr([0,0,1,0,0,0,0,0])
    # 4.4! 4.5!
    elif code == "4.6":
        comp_x = [comp for comp,names in comp_cr4_dict.items() if filename in names]
        if any([len([x for x in results[filename] if 'заключить сделки' in x\
                and any([comp in y for y in x])]) >= 2 for comp in comp_x]):
            numbers += Arr([1,0,1,0,0,0,0,0])
    #elif code == 4.7:
    elif code == "4.8":
        if ('продать компанию', '4') in tables[filename]:
            numbers += Arr([2,2,1,2,2,0,0,0])
    elif code == "4.9":
        if ('продать компанию', '4') in tables[filename] \
                and ('создать компанию', '4') in tables[filename][tables[filename].index(('продать компанию', '4')):]:
            numbers += Arr([1,0,0,0,1,0,0,0])
    elif code == "4.10":
        if ('продать компанию', '4') in tables[filename] \
                and any([y[y.find('инженер')+9:] in filename.lower() \
                for y in sum([x for x in results[filename] if 'нанять сотрудника' in x \
                and 'уровень игры: 4' in x], []) if y.startswith('инженер')]):
            numbers += Arr([-1,-1,-1,-1,-1,0,0,0])
    elif code == "4.11":
        comp_cr_total_x = [y[y.find('название')+10:] \
                for y in sum([x for x in results[filename] if 'создать компанию' in x], []) if y.startswith('название')]
        if len(comp_cr_total_x) > 1:
            if all([y[y.find(':')+2:] in comp_cr_total_x \
                   for y in sum([x for x in results[filename] if 'заключить сделки' in x], []) \
                   if any([y.startswith('от компании'), y.startswith('получатель')])]):
                numbers += Arr([0,0,0,0,0,0,4,0])
    elif code == "4.12":
        if any([y[y.find('покупатель')+12:] not in [filename[7:-5].lower()]+emp_hr_total_dict[filename] \
                for y in sum([x for x in results[filename] if 'купить продукт B2C' in x \
                and 'уровень игры: 3' in x], []) if y.startswith('покупатель')]) \
                and any([y[y.find('покупатель')+12:] not in [filename[7:-5].lower()]+emp_hr_total_dict[filename] \
                for y in sum([x for x in results[filename] if 'купить продукт B2C' in x \
                and 'уровень игры: 4' in x], []) if y.startswith('покупатель')]):
            numbers += Arr([0,1,0,1,0,0,0,0])
    elif code == "4.13":
        if any([y[y.find('ceo')+5:] not in [filename[7:-5].lower()]+emp_hr_total_dict[filename] \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x \
                and 'уровень игры: 4' not in x], []) if y.startswith('ceo')]) \
                and any([y[y.find('покупатель')+12:] not in [filename[7:-5].lower()]+emp_hr_total_dict[filename] \
                for y in sum([x for x in results[filename] if 'купить продукт B2C' in x \
                and 'уровень игры: 4' in x], []) if y.startswith('покупатель')]):
            numbers += Arr([0,0,0,1,1,0,0,0])
    elif code == "4.14":
        if any([y[y.find('ceo')+5:] not in filename.lower() \
                for y in sum([x for x in results[filename] if 'покупка готовой продукции' in x \
                and 'уровень игры: 4' not in x], []) if y.startswith('ceo')]):
            numbers += Arr([0,1,0,0,0,0,0,0])
            
    if filename in marks.keys():
        marks[filename] += numbers
    else:
        marks.update({filename: Arr([0]*8)})
        
    for key, value in marks.items():
        marks[key] = [min(y, 10) for y in value]
    
    return(1 if list(numbers) != [0]*8 else 0)

In [2]:
#os.chdir('p:///data')
#os.getcwd()

def main(path):
    os.chdir(path)
    global tables, results, scores
    tables, results, scores = {}, {}, {}
    all_values, marks = {}, {}
    files = [filename for filename in os.listdir() if filename.startswith('_игрок')]
    #for n in [1.1, 2.0, 2.6, 2.9, 3.4, 3.8, 3.9, 3.12, 4.0, 4.2]:
    codes = [f"1.{i}" for i in range(1,7)] + [f"2.{i}" for i in range(10)] + [f"3.{i}" for i in range(1,5)] \
            + [f"3.{i}" for i in range(7, 15)] + [f"4.{i}" for i in range(4)] + [f"4.{i}" for i in range(6, 15)]
    for filename in files:
        try:
            tables[filename], results[filename], scores[filename] = table_result_score(filename)
        except:
            pass
    for code in codes:
        all_values[code] = []
        for filename in files:
            try:
                all_values[code].append(rules(code, filename, marks))
            except Exception as e:
                #print(e)
                all_values[code].append(0)
                marks[filename] = [0]*8
                pass

    final_table = pd.DataFrame(all_values, index=[filename[7:-5] for filename in files])
    final_table['score'] = [list(x)[::-1][3:]+(list(x)[::-1][:3]) for x in marks.values()]
    
    return(final_table)

In [3]:
final_table = main('P:\\work\\data')
final_table.to_excel('final_table.xlsx')
final_table

,1.1,1.2,1.3,1.4,1.5,1.6,2.0,2.1,2.2,2.3,...,4.6,4.7,4.8,4.9,4.10,4.11,4.12,4.13,4.14,score
Аватар 1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0]"
Аватар 2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0]"
Аватар 3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0]"
Аватар 4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0]"
Аватар 5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0]"
Аватар 6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0]"
Аватар 7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0]"
Аватар 8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0]"
Акционер корпорации Х,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,"[2, 2, 1, 2, 2, 0, 0, 0]"
Андрей,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,"[2, 2, 1, 2, 2, 0, 0, 0]"


In [4]:
df_t = pd.DataFrame()
key_actions = {'k1': [], 'k2': [], 'k3': [], 'k4': [], 'k5': []}

for filename in tables.keys():
    if final_table.at[filename[7:-5], '1.3']:
        key_actions['k1'].append('Проинвестировал в повышение производительности инженерного труда')
    elif final_table.at[filename[7:-5], '1.1']:
        key_actions['k1'].append('Получил приз по результатам конкурса инженерных решений')
    elif final_table.at[filename[7:-5], '1.4']:
        key_actions['k1'].append('Успешно идентифицировал возможность заработка в качестве сотрудника корпорации')
    elif filename in sum(comp_cr2_dict.values(), []):
        key_actions['k1'].append('При первой возможности перешёл в предпринимательскую позицию')
    else:
        key_actions['k1'].append('Пропуск')
        
for filename in tables.keys():
    if final_table.at[filename[7:-5], '2.7']:
        key_actions['k2'].append('Прошёл квалификацию на роль поставщика крупной корпорации')
    elif tables[filename].count(('заключить сделки', '3')) >2 and not final_table.at[filename[7:-5], '1.4']:
        key_actions['k2'].append('Смог определить рамки и сконструировать деятельность компании' \
        'в условиях высокой неопределённости и отсутствия внешнего заказчика')
    elif filename in sum(comp_cr2_dict.values(), []):
        key_actions['k2'].append('При первой возможности перешёл в предпринимательскую позицию')
    elif final_table.at[filename[7:-5], '1.4']:
        key_actions['k2'].append('Успешно освоил рамки поставленной старшим партнёром позиции')
    else:
        key_actions['k2'].append('Пропуск')    

for index, filename in enumerate(tables.keys()):
    if tables[filename].count(('заключить сделки', '3')) >2 and not final_table.at[filename[7:-5], '1.4'] \
            and key_actions['k2'][index] != 'Смог определить рамки и сконструировать деятельность компании' \
            'в условиях высокой неопределённости и отсутствия внешнего заказчика':
        key_actions['k3'].append('Смог определить рамки и сконструировать деятельность компании' \
        'в условиях высокой неопределённости и отсутствия внешнего заказчика')
    elif any([y[y.find('инвестор')+10:] not in filename.lower() \
            for y in sum([x for x in results[filename] if 'инвестировать' in x], []) if y.startswith('инвестор')]):
        key_actions['k3'].append('Успешно провёл переговоры с инвестором и получил инвестиции')
    elif final_table.at[filename[7:-5], '1.4'] and len([x for x in results[filename] if 'заключить сделки' in x \
            and 'тип договора: продажа инструмента' not in x and 'уровень игры: 3' in x]) >1:
        key_actions['k3'].append('Выполнил задание старшего партнёра на строительство производства')
    else:
        key_actions['k3'].append('Пропуск')
        
for index, filename in enumerate(tables.keys()):
    if final_table.at[filename[7:-5], '4.14']:
        key_actions['k4'].append('Организовал производство крупной партии продукции по заказу старшего партнёра')
    elif final_table.at[filename[7:-5], '4.7']:
        key_actions['k4'].append('Корректно идентифицировал и занял нишу в системе разделения труда')
    elif any([y[y.find('инвестор')+10:] not in filename.lower() \
            for y in sum([x for x in results[filename] if 'инвестировать' in x], []) if y.startswith('инвестор')]) \
            and key_actions['k3'][index] != 'Успешно провёл переговоры с инвестором и получил инвестиции':
        key_actions['k4'].append('Успешно провёл переговоры с инвестором и получил инвестиции')
    elif final_table.at[filename[7:-5], '4.8']:
        key_actions['k4'].append('Продал построенную компанию')
    else:
        key_actions['k4'].append('Пропуск')
        
for index, filename in enumerate(tables.keys()):
    if final_table.at[filename[7:-5], '4.8'] and key_actions['k4'][index] != 'Продал построенную компанию':
        key_actions['k5'].append('Продал построенную компанию')
    elif ('найм в качестве директора компании', '4') in tables[filename] and ('продать компанию', '4') in tables[filename] \
            and ('заключить сделки', '4') in tables[filename][tables[filename].index(('продать компанию', '4')):]:
        key_actions['k5'].append('После продажи компании принял предложение о работе наёмным директором \
                и подтвердил эффективность проданной компании')
    elif final_table.at[filename[7:-5], '4.9']:
        key_actions['k5'].append('После продажи компании открыл новую компанию')
    else:
        key_actions['k5'].append('Пропуск')
        
df_t = pd.DataFrame(key_actions, final_table.index)
df_t.to_excel('table_keys.xlsx')
df_t

,k1,k2,k3,k4,k5
Аватар 1,Пропуск,Пропуск,Пропуск,Пропуск,Пропуск
Аватар 2,Пропуск,Пропуск,Пропуск,Пропуск,Пропуск
Аватар 3,Пропуск,Пропуск,Пропуск,Пропуск,Пропуск
Аватар 4,Пропуск,Пропуск,Пропуск,Пропуск,Пропуск
Аватар 5,Пропуск,Пропуск,Пропуск,Пропуск,Пропуск
Аватар 6,Пропуск,Пропуск,Пропуск,Пропуск,Пропуск
Аватар 7,Пропуск,Пропуск,Пропуск,Пропуск,Пропуск
Аватар 8,Пропуск,Пропуск,Пропуск,Пропуск,Пропуск
Акционер корпорации Х,Пропуск,Пропуск,Пропуск,Продал построенную компанию,Пропуск
Андрей,Пропуск,Пропуск,Пропуск,Продал построенную компанию,Пропуск
